In [15]:
import os
import csv
import json
import pandas as pd
import numpy as np

In [16]:
df_10k = pd.read_csv("../data/topic_10k.csv", encoding="utf-8", index_col=0)

In [17]:
df_10k.head()

,interest_slug,story_id,title,description,date,url,lang,paywall,source_name,source_slug,...,source_lat,source_lon,source_place,article_file,article_title,article_description,article_image_url,article_text,article_date_publish,article_authors
0,california-politics,ec08507a-0644-497a-8933-857281f6a95e,Los Angeles wildfires death toll rises as crew...,LOS ANGELES (AP) — The death toll from the wil...,2025-01-12T14:06:00.000Z,https://wtop.com/national/2025/01/los-angeles-...,en,no,Washington Top News,washington-top-news,...,38.897679,-77.051110,"Washington, District of Columbia, United States",data/topics/json/416788.json,24 dead as fire crews try to corral Los Angele...,Firefighters scrambled Sunday to make further ...,https://wtop.com/national/2025/01/los-angeles-...,The National Weather Service issued red flag w...,2025-01-12 10:30:17,The Associated Press
1,big-ten,96c4a21f-2008-4d50-b29e-67ad8728dd31,Twelve-team playoff not big enough to avoid co...,"The new, 12-team College Football Playoff brin...",2024-12-08T22:40:20.000Z,https://www.limaohio.com/sports/2024/12/08/twe...,en,no,The Lima News,the-lima-news,...,34.234545,-84.482904,"Cherokee County, Georgia, United States",data/topics/json/25427.json,team playoff not big enough to avoid controver...,"The new, 12-team College Football Playoff brin...",https://www.limaohio.com/wp-content/uploads/20...,"The new, 12-team College Football Playoff brin...",2024-12-08 22:40:20,Associated Press; www.limaohio.com
2,california-united-states,20855826-624f-4796-86c1-3bea0fd00649,LA mayor dismisses fire chief over response to...,"Six weeks after devastating wildfires, Los Ang...",2025-02-21T20:27:00.000Z,https://www.wdsu.com/article/los-angeles-mayor...,en,no,WDSU,wdsu,...,29.951430,-90.071742,"New Orleans, Orleans Parish, Louisiana, United...",data/topics/json/56984.json,LA Mayor Ousts Fire Chief Over Wildfire,Mayor Karen Bass removes Fire Chief Kristin Cr...,https://kubrick.htvapps.com/htv-prod-media.s3....,that doesn't live there. Lynn McIntyre is one ...,2025-02-21 20:28:00,MICHAEL BLOOD
3,gaming_d04843,35df203e-aeae-4bc4-86f2-d4d4a588c392,"Jason Momoa to play Lobo in new DC Universe, w...",The DC universe has cast another major role. J...,2024-12-30T10:18:14.000Z,https://www.digitaltrends.com/movies/jason-mom...,en,no,Digital Trends,digital-trends,...,35.642134,-92.654776,United States,data/topics/json/325673.json,"Jason Momoa to play Lobo in new DC Universe, w...",The DC universe has cast another major role. J...,https://www.digitaltrends.com/wp-content/uploa...,Aquaman has left the building.\nAfter starring...,2024-12-30 10:18:14,Dan Girolamo
4,2024-uk-election,370af72e-6a83-44a1-be1f-93175c43ab5f,UK Prime Minister Commits to Deploying Troops ...,"LONDON, ENGLAND (VINnews)-Prime Minister Keir ...",2025-02-17T12:40:57.000Z,https://vinnews.com/2025/02/17/uk-prime-minist...,en,no,VINnews,vinnews,...,NaN,NaN,NaN,data/topics/json/66043.json,UK Prime Minister Commits to Deploying Troops ...,"LONDON, ENGLAND (VINnews)-Prime Minister Keir ...",https://vinnews.com/wp-content/uploads/2025/02...,"LONDON, ENGLAND (VINnews)-Prime Minister Keir ...",2025-02-20 00:00:00,Reuven Rosenfeld


In [18]:
jsonl_dir = "../data/topics_10k_logits/deepseek"

In [19]:
def parse_logits(logits):
    return np.array([
        logits["Left"],
        logits["Left-Center"],
        logits["Center"],
        logits["Center-Right"],
        logits["Right"]
    ])

label_mapping = {
    "left": 0,
    "left-center": 1,
    "center": 2,
    "center-right": 3,
    "right": 4
}

data = []

for file in os.listdir(jsonl_dir):
    with open(os.path.join(jsonl_dir, file), "r", encoding="utf-8") as f:
        for line in f:
            entry = json.loads(line)
            data.append(entry)

df_logits = pd.DataFrame.from_records(data, index="doc_id")
df_logits["logit_scores"] = df_logits["logit_scores"].apply(parse_logits)
df_logits["softmax_probabilities"] = df_logits["softmax_probabilities"].apply(parse_logits)
df_logits["label"] = df_logits["label"].map(label_mapping)

In [20]:
df_logits.head()

,url,reasoning,logit_scores,softmax_probabilities,label
doc_id,,,,,
13,https://www.upi.com/Top_News/US/2025/02/11/DOJ...,```markdown\nReasoning:\n\n- The article frame...,"[-3.2, -1.5, 0.8, 2.1, 4.7]","[0.0123, 0.0369, 0.0864, 0.1678, 0.6966]",4
3,https://www.digitaltrends.com/movies/jason-mom...,```markdown\nReasoning:\n\n- The article focus...,"[-1.25, -0.75, 2.1, -0.85, -1.35]","[0.0412, 0.0635, 0.7023999999999999, 0.0588999...",2
10,https://www.rnz.co.nz/news/world/538075/cybert...,```markdown\nReasoning:\n- The article repeate...,"[2.75, 4.25, 3.5, 1.25, 0.75]","[0.2048, 0.391, 0.2634, 0.0842, 0.056600000000...",1
14,https://www.rnz.co.nz/news/world/537413/trump-...,```markdown\nReasoning:\n\n- **Subjective phra...,"[-5.0, -2.0, 1.0, 3.5, 8.0]","[0.0007000000000000001, 0.0019, 0.0087, 0.0412...",4
9,https://www.theglobeandmail.com/world/article-...,```markdown\nReasoning:\n\n- **Climate change ...,"[1.25, 2.75, 3.5, 0.75, -0.5]","[0.12539999999999998, 0.2532, 0.4789, 0.1173, ...",2


In [21]:
df_merged = df_logits.merge(df_10k, how="left")

In [22]:
df_merged

,url,reasoning,logit_scores,softmax_probabilities,label,interest_slug,story_id,title,description,date,...,source_lat,source_lon,source_place,article_file,article_title,article_description,article_image_url,article_text,article_date_publish,article_authors
0,https://www.upi.com/Top_News/US/2025/02/11/DOJ...,```markdown\nReasoning:\n\n- The article frame...,"[-3.2, -1.5, 0.8, 2.1, 4.7]","[0.0123, 0.0369, 0.0864, 0.1678, 0.6966]",4,eric-adams_a02992,293d4278-2709-4acc-9cd2-e9d9384d6c82,DOJ moves to dismiss case against NYC Mayor Er...,The U.S. Justice Department has directed prose...,2025-02-11T03:50:42.000Z,...,38.905563,-77.040467,"Washington, District of Columbia, United States",data/topics/json/163014.json,DOJ moves to dismiss case against NYC Mayor Er...,The U.S. Justice Department has directed prose...,https://cdnph.upi.com/sv/ph/og/upi/15617392587...,Feb. 11 (UPI) -- The U.S. Justice Department h...,2025-02-11 03:50:42,Darryl Coote
1,https://www.digitaltrends.com/movies/jason-mom...,```markdown\nReasoning:\n\n- The article focus...,"[-1.25, -0.75, 2.1, -0.85, -1.35]","[0.0412, 0.0635, 0.7023999999999999, 0.0588999...",2,gaming_d04843,35df203e-aeae-4bc4-86f2-d4d4a588c392,"Jason Momoa to play Lobo in new DC Universe, w...",The DC universe has cast another major role. J...,2024-12-30T10:18:14.000Z,...,35.642134,-92.654776,United States,data/topics/json/325673.json,"Jason Momoa to play Lobo in new DC Universe, w...",The DC universe has cast another major role. J...,https://www.digitaltrends.com/wp-content/uploa...,Aquaman has left the building.\nAfter starring...,2024-12-30 10:18:14,Dan Girolamo
2,https://www.rnz.co.nz/news/world/538075/cybert...,```markdown\nReasoning:\n- The article repeate...,"[2.75, 4.25, 3.5, 1.25, 0.75]","[0.2048, 0.391, 0.2634, 0.0842, 0.056600000000...",1,new-year,7a04aade-5f41-4fc5-8573-3abbb4552acb,Cybertruck explosion not linked to New Orleans...,Law enforcement officials reportedly identifie...,2025-01-02T20:12:23.000Z,...,-40.776605,174.860210,New Zealand,data/topics/json/14862.json,Cybertruck explosion not linked to New Orleans...,Law enforcement officials reportedly identifie...,https://media.rnztools.nz/rnz/image/upload/s--...,"By Ronda Churchill, David Shepardson and Karen...",2025-01-03 09:12:23,RNZ News
3,https://www.rnz.co.nz/news/world/537413/trump-...,```markdown\nReasoning:\n\n- **Subjective phra...,"[-5.0, -2.0, 1.0, 3.5, 8.0]","[0.0007000000000000001, 0.0019, 0.0087, 0.0412...",4,latin-america,c34d413d-b030-4419-bf8c-af4da684d0ef,Trump says he might demand Panama hand over canal,The President-elect has accused Panama of char...,2024-12-22T05:45:39.000Z,...,-40.051143,173.308824,New Zealand,data/topics/json/45477.json,Trump says he might demand Panama hand over canal,The President-elect has accused Panama of char...,https://media.rnztools.nz/rnz/image/upload/s--...,"By Gram Slattery, Reuters\nPresident-elect Don...",2024-12-22 18:45:39,RNZ News
4,https://www.theglobeandmail.com/world/article-...,```markdown\nReasoning:\n\n- **Climate change ...,"[1.25, 2.75, 3.5, 0.75, -0.5]","[0.12539999999999998, 0.2532, 0.4789, 0.1173, ...",2,national-weather-service_afba2f,626cff37-263d-428e-acc4-9649edbcd978,Heaviest snowfall in a decade possible as wint...,Snow and ice blanketed major roadways in parts...,2025-01-05T14:19:04.000Z,...,55.128569,-107.645033,Canada,data/topics/json/94086.json,Heaviest snowfall in a decade possible as wint...,Snow and ice blanketed major roadways in parts...,https://www.theglobeandmail.com/resizer/v2/QO6...,"A blast of snow, ice, wind and plunging temper...",2025-01-05 14:19:04,Heather Hollingsworth And Brian Witte
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7975,https://www.protothema.gr/world/article/158930...,```markdown\nReasoning:\n\n- **Content Mismatc...,"[-1.2, -0.8, 2.5, -1.0, -1.5]","[0.0625, 0.0833, 0.6597, 0.0729, 0.1216]",2,iran,4c15810b-c677-4d1b-a72f-25b190d35291,Vladimir Putin signs 20-year Russia-Iran strat.

In [23]:
df_merged.to_csv("../data/topics_10k_deepseek_logits.csv", quoting=csv.QUOTE_NONNUMERIC)

In [24]:
df_merged[["logit_scores", "softmax_probabilities", "label", "title", "article_description", "article_text"]].to_json("../data/topics_10k_deepseek_logits.json", orient="records")

## No logits

In [12]:
import csv
import json
import pandas as pd
import numpy as np

In [13]:
df_10k = pd.read_csv("../data/topic_10k.csv", encoding="utf-8", index_col=0)

In [14]:
df_10k.head()

,interest_slug,story_id,title,description,date,url,lang,paywall,source_name,source_slug,...,source_lat,source_lon,source_place,article_file,article_title,article_description,article_image_url,article_text,article_date_publish,article_authors
0,california-politics,ec08507a-0644-497a-8933-857281f6a95e,Los Angeles wildfires death toll rises as crew...,LOS ANGELES (AP) — The death toll from the wil...,2025-01-12T14:06:00.000Z,https://wtop.com/national/2025/01/los-angeles-...,en,no,Washington Top News,washington-top-news,...,38.897679,-77.051110,"Washington, District of Columbia, United States",data/topics/json/416788.json,24 dead as fire crews try to corral Los Angele...,Firefighters scrambled Sunday to make further ...,https://wtop.com/national/2025/01/los-angeles-...,The National Weather Service issued red flag w...,2025-01-12 10:30:17,The Associated Press
1,big-ten,96c4a21f-2008-4d50-b29e-67ad8728dd31,Twelve-team playoff not big enough to avoid co...,"The new, 12-team College Football Playoff brin...",2024-12-08T22:40:20.000Z,https://www.limaohio.com/sports/2024/12/08/twe...,en,no,The Lima News,the-lima-news,...,34.234545,-84.482904,"Cherokee County, Georgia, United States",data/topics/json/25427.json,team playoff not big enough to avoid controver...,"The new, 12-team College Football Playoff brin...",https://www.limaohio.com/wp-content/uploads/20...,"The new, 12-team College Football Playoff brin...",2024-12-08 22:40:20,Associated Press; www.limaohio.com
2,california-united-states,20855826-624f-4796-86c1-3bea0fd00649,LA mayor dismisses fire chief over response to...,"Six weeks after devastating wildfires, Los Ang...",2025-02-21T20:27:00.000Z,https://www.wdsu.com/article/los-angeles-mayor...,en,no,WDSU,wdsu,...,29.951430,-90.071742,"New Orleans, Orleans Parish, Louisiana, United...",data/topics/json/56984.json,LA Mayor Ousts Fire Chief Over Wildfire,Mayor Karen Bass removes Fire Chief Kristin Cr...,https://kubrick.htvapps.com/htv-prod-media.s3....,that doesn't live there. Lynn McIntyre is one ...,2025-02-21 20:28:00,MICHAEL BLOOD
3,gaming_d04843,35df203e-aeae-4bc4-86f2-d4d4a588c392,"Jason Momoa to play Lobo in new DC Universe, w...",The DC universe has cast another major role. J...,2024-12-30T10:18:14.000Z,https://www.digitaltrends.com/movies/jason-mom...,en,no,Digital Trends,digital-trends,...,35.642134,-92.654776,United States,data/topics/json/325673.json,"Jason Momoa to play Lobo in new DC Universe, w...",The DC universe has cast another major role. J...,https://www.digitaltrends.com/wp-content/uploa...,Aquaman has left the building.\nAfter starring...,2024-12-30 10:18:14,Dan Girolamo
4,2024-uk-election,370af72e-6a83-44a1-be1f-93175c43ab5f,UK Prime Minister Commits to Deploying Troops ...,"LONDON, ENGLAND (VINnews)-Prime Minister Keir ...",2025-02-17T12:40:57.000Z,https://vinnews.com/2025/02/17/uk-prime-minist...,en,no,VINnews,vinnews,...,NaN,NaN,NaN,data/topics/json/66043.json,UK Prime Minister Commits to Deploying Troops ...,"LONDON, ENGLAND (VINnews)-Prime Minister Keir ...",https://vinnews.com/wp-content/uploads/2025/02...,"LONDON, ENGLAND (VINnews)-Prime Minister Keir ...",2025-02-20 00:00:00,Reuven Rosenfeld


In [15]:
jsonl_files = [
    "../data/topics_10k/topics_10k_ideology_20250308_154650.jsonl",
    "../data/topics_10k/topics_10k_ideology_20250308_175522.jsonl"
]

In [16]:
label_mapping = {
    "left": 0,
    "left-center": 1,
    "center": 2,
    "center-right": 3,
    "right": 4
}

data = []

for file in jsonl_files:
    with open(file, "r", encoding="utf-8") as f:
        for line in f:
            entry = json.loads(line)
            data.append(entry)

df_logits = pd.DataFrame.from_records(data, index="doc_id")
df_logits["label"] = df_logits["label"].map(label_mapping)

In [17]:
df_logits.head()

,score,label,url,reasoning
doc_id,,,,
3,0.0,2,https://www.digitaltrends.com/movies/jason-mom...,```markdown\nReasoning:\n\n- **Potentially Bia...
4,0.8,3,https://vinnews.com/2025/02/17/uk-prime-minist...,```markdown\nReasoning:\n\n- **Potentially Bia...
8,-0.5,1,https://ny1.com/nyc/all-boroughs/business/2025...,```markdown\nReasoning:\n\n**Potentially Biase...
9,-0.1,2,https://www.theglobeandmail.com/world/article-...,```markdown\nReasoning:\n\n- **Potentially Bia...
13,1.4,3,https://www.upi.com/Top_News/US/2025/02/11/DOJ...,```markdown\nReasoning:\n\n- **Potentially Bia...


In [18]:
df_merged = df_logits.merge(df_10k, how="left")

In [19]:
df_merged

,score,label,url,reasoning,interest_slug,story_id,title,description,date,lang,...,source_lat,source_lon,source_place,article_file,article_title,article_description,article_image_url,article_text,article_date_publish,article_authors
0,0.0,2,https://www.digitaltrends.com/movies/jason-mom...,```markdown\nReasoning:\n\n- **Potentially Bia...,gaming_d04843,35df203e-aeae-4bc4-86f2-d4d4a588c392,"Jason Momoa to play Lobo in new DC Universe, w...",The DC universe has cast another major role. J...,2024-12-30T10:18:14.000Z,en,...,35.642134,-92.654776,United States,data/topics/json/325673.json,"Jason Momoa to play Lobo in new DC Universe, w...",The DC universe has cast another major role. J...,https://www.digitaltrends.com/wp-content/uploa...,Aquaman has left the building.\nAfter starring...,2024-12-30 10:18:14,Dan Girolamo
1,0.8,3,https://vinnews.com/2025/02/17/uk-prime-minist...,```markdown\nReasoning:\n\n- **Potentially Bia...,2024-uk-election,370af72e-6a83-44a1-be1f-93175c43ab5f,UK Prime Minister Commits to Deploying Troops ...,"LONDON, ENGLAND (VINnews)-Prime Minister Keir ...",2025-02-17T12:40:57.000Z,en,...,NaN,NaN,NaN,data/topics/json/66043.json,UK Prime Minister Commits to Deploying Troops ...,"LONDON, ENGLAND (VINnews)-Prime Minister Keir ...",https://vinnews.com/wp-content/uploads/2025/02...,"LONDON, ENGLAND (VINnews)-Prime Minister Keir ...",2025-02-20 00:00:00,Reuven Rosenfeld
2,-0.5,1,https://ny1.com/nyc/all-boroughs/business/2025...,```markdown\nReasoning:\n\n**Potentially Biase...,german-politics,58482f95-c635-45b8-bdd8-d7631a8b3f6f,"SEC sues Elon Musk, saying he didn't disclose ...","As a result, the SEC alleges, Musk was able to...",2025-01-15T00:51:00.000Z,en,...,40.701546,-73.987628,"New York, New York, United States",data/topics/json/24198.json,"SEC sues Elon Musk, saying he didn't disclose ...","As a result, the SEC alleges, Musk was able to...",https://s7d2.scene7.com/is/image/TWCNews/elon_...,The U.S. Securities and Exchange Commission ha...,2025-01-15 00:00:00,Associated Press
3,-0.1,2,https://www.theglobeandmail.com/world/article-...,```markdown\nReasoning:\n\n- **Potentially Bia...,national-weather-service_afba2f,626cff37-263d-428e-acc4-9649edbcd978,Heaviest snowfall in a decade possible as wint...,Snow and ice blanketed major roadways in parts...,2025-01-05T14:19:04.000Z,en,...,55.128569,-107.645033,Canada,data/topics/json/94086.json,Heaviest snowfall in a decade possible as wint...,Snow and ice blanketed major roadways in parts...,https://www.theglobeandmail.com/resizer/v2/QO6...,"A blast of snow, ice, wind and plunging temper...",2025-01-05 14:19:04,Heather Hollingsworth And Brian Witte
4,1.4,3,https://www.upi.com/Top_News/US/2025/02/11/DOJ...,```markdown\nReasoning:\n\n- **Potentially Bia...,eric-adams_a02992,293d4278-2709-4acc-9cd2-e9d9384d6c82,DOJ moves to dismiss case against NYC Mayor Er...,The U.S. Justice Department has directed prose...,2025-02-11T03:50:42.000Z,en,...,38.905563,-77.040467,"Washington, District of Columbia, United States",data/topics/json/163014.json,DOJ moves to dismiss case against NYC Mayor Er...,The U.S. Justice Department has directed prose...,https://cdnph.upi.com/sv/ph/og/upi/15617392587...,Feb. 11 (UPI) -- The U.S. Justice Department h...,2025-02-11 03:50:42,Darryl Coote
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3987,-0.8,1,https://koreatimes.co.kr/www/nation/2024/12/50...,```markdown\nReasoning:\n\n- **Potentially Bia...,2024-france-election,e835aa35-353e-4898-a6a6-fd0e10819ec8,Gisele Pelicot's ex-husband found guilty of ra...,A court in France on Thursday sentenced the ex...,2024-12-19T11:56:00.000Z,en,...,36.268320,127.885446,South Korea,data/topics/json/51196.json,Gisele Pelicot's ex-husband found guilty of ra...,A court in France on Thursday sentenced the ex...,https://img.koreatimes.co.kr/upload/thumbnailV...,A court in France on Thursday sentenced the ex...,2024-12-19 20:56:00,NaN
3988,0.5,3,https://www.usnews.com/news/world/

In [20]:
df_merged.to_csv("../data/topics_10k_deepseek_5_class.csv", quoting=csv.QUOTE_NONNUMERIC)

## Peft Weights

In [12]:
import os
import json

import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import torch
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    get_linear_schedule_with_warmup
)
from peft import (
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model,
    load_peft_weights,
    set_peft_model_state_dict
)
from peft import PeftModel
from accelerate import Accelerator
import torch.nn.functional as F  # For KL divergence and softmax

In [5]:
state_dict = load_peft_weights("best_model/best_model_r16_b4")

In [6]:
state_dict

{'base_model.model.model.embed_tokens.weight': tensor([[ 0.0045,  0.0166,  0.0210,  ..., -0.0054, -0.0422, -0.0315],
         [ 0.0215, -0.0238,  0.0211,  ..., -0.0107, -0.0011, -0.0374],
         [ 0.0136,  0.0104,  0.0128,  ...,  0.0081, -0.0122,  0.0051],
         ...,
         [ 0.0009,  0.0164, -0.0193,  ..., -0.0003, -0.0030,  0.0066],
         [ 0.0009,  0.0164, -0.0193,  ..., -0.0003, -0.0030,  0.0066],
         [ 0.0028,  0.0081, -0.0152,  ..., -0.0019, -0.0018,  0.0045]],
        device='cuda:0'),
 'base_model.model.model.layers.0.mlp.down_proj.lora_A.weight': tensor([[-0.0032, -0.0332, -0.0115,  ..., -0.0299,  0.0187, -0.0286],
         [ 0.0275, -0.0069, -0.0182,  ...,  0.0040, -0.0145, -0.0012],
         [ 0.0259,  0.0256, -0.0130,  ...,  0.0284, -0.0012, -0.0225],
         ...,
         [ 0.0056, -0.0119,  0.0021,  ..., -0.0070, -0.0077,  0.0265],
         [ 0.0178, -0.0298, -0.0098,  ...,  0.0143,  0.0051,  0.0068],
         [-0.0069, -0.0394,  0.0305,  ...,  0.0076,  0.

In [11]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model_name = "meta-llama/Llama-3.2-1B"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, token=HF_TOKEN)

# Load model with quantization config and task-specific number of labels
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, 
    quantization_config=bnb_config, 
    num_labels=5,
    token=HF_TOKEN
)

# Set pad token and resize embeddings
tokenizer.add_special_tokens({"pad_token": "[PAD]"})
model.config.pad_token_id = tokenizer.pad_token_id
model.resize_token_embeddings(len(tokenizer))

# Prepare model for efficient training
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

# Setup LoRA configuration for QLoRA training
lora_config = LoraConfig(
    r=16,
    lora_alpha=64,
    lora_dropout=0.1,
    bias="none",
    target_modules=[
        "down_proj",
        "o_proj",
        "q_proj",
        "k_proj",
        "gate_proj",
        "up_proj",
        "v_proj"
    ],
    task_type="SEQ_CLS"
)


model = get_peft_model(model, lora_config)

trainable_params = [n for n, p in model.named_parameters() if p.requires_grad]
print("Trainable parameters:", trainable_params)

model.get_model_status()

`low_cpu_mem_usage` was None, now default to True since model is quantized.
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters: ['base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight', 'base_model.model.model.layers.0.mlp.gate_proj.lora_A.default.weight', 'base_model.model.model.layers.0.mlp.gate_proj.lora_B.default.weight', 'base_model.model.model.layers.0.mlp.up_proj.lora_A.default.weight', 'base_model.model.model.layers.0.mlp.up_proj.lora_B.default.weight', 'base_model.model.model.layers.0.mlp.down_proj.lora_A.default.weight', 'base_model.model.model.layers.0.m

TunerModelStatus(base_model_type='LlamaForSequenceClassification', adapter_model_type='LoraModel', peft_types={'default': 'LORA'}, trainable_params=11282432, total_params=1247109120, num_adapter_layers=112, enabled=True, active_adapters=['default'], merged_adapters=[], requires_grad={'default': True}, available_adapters=['default'], devices={'default': ['cuda']})

In [13]:
set_peft_model_state_dict(
    model,
    state_dict,
    adapter_name="default",
    ignore_mismatched_sizes=True,
)

/home/ubuntu/.local/lib/python3.10/site-packages/peft/utils/save_and_load.py:473: UserWarning: Some weights of PeftModelForSequenceClassification were not initialized from the model checkpoint and are being ignored because you passed `ignore_mismatched_sizes=True`: - base_model.model.score.modules_to_save.default.weight: found shape torch.Size([3, 2048]) in the checkpoint and torch.Size([5, 2048]) in the model instantiated.
  warnings.warn(msg)


_IncompatibleKeys(missing_keys=['base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight', 'base_model.model.model.layers.0.self_attn.k_proj.base_layer.weight', 'base_model.model.model.layers.0.self_attn.v_proj.base_layer.weight', 'base_model.model.model.layers.0.self_attn.o_proj.base_layer.weight', 'base_model.model.model.layers.0.mlp.gate_proj.base_layer.weight', 'base_model.model.model.layers.0.mlp.up_proj.base_layer.weight', 'base_model.model.model.layers.0.mlp.down_proj.base_layer.weight', 'base_model.model.model.layers.0.input_layernorm.weight', 'base_model.model.model.layers.0.post_attention_layernorm.weight', 'base_model.model.model.layers.1.self_attn.q_proj.base_layer.weight', 'base_model.model.model.layers.1.self_attn.k_proj.base_layer.weight', 'base_model.model.model.layers.1.self_attn.v_proj.base_layer.weight', 'base_model.model.model.layers.1.self_attn.o_proj.base_layer.weight', 'base_model.model.model.layers.1.mlp.gate_proj.base_layer.weight', 'base_model.model

In [14]:
trainable_params = [n for n, p in model.named_parameters() if p.requires_grad]
print("Trainable parameters:", trainable_params)

Trainable parameters: ['base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight', 'base_model.model.model.layers.0.mlp.gate_proj.lora_A.default.weight', 'base_model.model.model.layers.0.mlp.gate_proj.lora_B.default.weight', 'base_model.model.model.layers.0.mlp.up_proj.lora_A.default.weight', 'base_model.model.model.layers.0.mlp.up_proj.lora_B.default.weight', 'base_model.model.model.layers.0.mlp.down_proj.lora_A.default.weight', 'base_model.model.model.layers.0.m